In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import FIRE as fire
import imp
imp.reload(fire)

In [ ]:
# Boundary conditions / external effects
all_conditions = fire.FIREExternalConditionsArray('../External.json')
print(f"Available conditions: {all_conditions.keys()}")
conditions = all_conditions.select("SCENARIO_AVERAGE_INFLATION_AND_CAPITAL_TAX")

# Income/assets + Investments
all_investments = fire.FIREInvestmentArray('../Investments.json')
investments = all_investments.select("SCENARIO_1")
print(f"Available investments: {all_investments.keys()}")

# Target
all_targets = fire.FIRETargetArray('../Targets.json')
#target = all_targets.select("SCENARIO_LIVABLE_AGGRESSIVE")
print(f"Available targets: {all_targets.keys()}")

# Retirement ages in varying scenarios

In [ ]:
# Build up simulations

all_sims = dict()
all_sims_value = dict()
for tk, tv in all_targets.items():
    simulation = fire.FIRESimulation(investments=investments, target=tv, conditions=conditions)
    simulation.perform()
    all_sims[tk] = simulation.simulation_retirement_age
    all_sims_value[tk] = tv.calc_target_value()

In [ ]:
# Plot retirement ages
all_k = [*all_sims.keys()]
all_v = np.array([*all_sims.values()])

plt.figure(figsize=(16, 4))
plt.bar([k.replace("_","_\n") for k in all_k], all_v)
plt.xlabel("Scenario name")
plt.ylabel("Retirement age / [years]")
for ii,(d,v) in enumerate(zip(all_k, all_v)):
    col = "black"
    plt.text(ii, v,f"{v} years", ha = 'center', va='bottom', c=col)

In [ ]:
# Plot target values
all_k = [*all_sims_value.keys()]
all_v = np.array([*all_sims_value.values()])

plt.figure(figsize=(16, 4))
plt.bar([k.replace("_","_\n") for k in all_k], all_v)
plt.xlabel("Scenario name")
plt.ylabel("Target value / [€]")
for ii,(d,v) in enumerate(zip(all_k, all_v)):
    col = "black"
    plt.text(ii, v,f"{v}€", ha = 'center', va='bottom', c=col)

# Influence of monthly investment on retirement age

In [ ]:
# Build up simulations
def simulate_investments(scenario, all_rates):
    all_sims = dict()
    for r in all_rates:
        investments.investment_monthly = r
        simulation = fire.FIRESimulation(investments=investments, target=all_targets[scenario], conditions=conditions)
        simulation.perform()
        all_sims[r] = simulation.simulation_retirement_age
    return all_sims

# Plotting
def plot_investments_vs_retirement_age():
    all_rates = np.arange(1000,6000,500)
    net_savings_share, income_tax_rate = 0.45, 0.5
    all_est_incomes = all_rates * 12 / net_savings_share / (1.-income_tax_rate)
    for sc in all_targets.keys():
        all_sims = simulate_investments(sc, all_rates)
        all_k = [*all_sims.keys()]
        all_v = np.array([*all_sims.values()])
        plt.plot(all_k, all_v, label = sc)
        
    plt.title("Influence of monthly investments on retirement age")
    plt.xlabel("Monthly investments / [€]")
    plt.ylabel("Retirement age / [years]")
    plt.legend()
    ax2 = plt.twiny()
    ax2.plot(all_est_incomes, all_v, alpha = 0)
    ax2.set_xlabel("Estimated yearly income / [€]")

In [ ]:
plot_investments_vs_retirement_age()

In [ ]:
!jupyter nbconvert --to pdf FIRE_Scenario_Variation.ipynb --output Report_Scenario_Variation-xyz.pdf